In [37]:
# loan_model_pipeline.py
# Run as a script or in notebook cells. Requires: pandas, sqlalchemy, pyodbc, scikit-learn, joblib

import os
import getpass
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
)
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
import joblib
from sqlalchemy import create_engine, text


In [39]:
import urllib
from sqlalchemy import create_engine, text

# Connection config
DRIVER = "ODBC Driver 17 for SQL Server"
SERVER = "localhost"  # your default instance
DATABASE = "loan_analytics_dev"

# Build connection string
odbc_str = f"DRIVER={{{DRIVER}}};SERVER={SERVER};DATABASE={DATABASE};Trusted_Connection=yes;"
conn_str = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(odbc_str)}"

# Create SQLAlchemy engine
engine = create_engine(conn_str, fast_executemany=True)

# Test query (no tables required!)
with engine.connect() as conn:
    result = conn.execute(text("SELECT DB_NAME() AS ConnectedTo;"))
    print(result.fetchall())


[('loan_analytics_dev',)]


In [41]:
# Step 1: load vw_CleanLoans
sql = "SELECT * FROM dbo.vw_CleanLoans;"
print("Loading vw_CleanLoans ...")
df = pd.read_sql(sql, engine)
print("Rows loaded:", len(df))
print(df.head())

Loading vw_CleanLoans ...
Rows loaded: 1000
   ApplicationID  CustomerID ApplicationDate  Age   Income  EmploymentYears  \
0           2001        1494      2023-04-20   54  2205134               10   
1           2002        1124      2024-09-18   33  2955165                8   
2           2003        1333      2023-05-19   30  1592245                7   
3           2004        1380      2024-10-05   38  2528538               19   
4           2005        1406      2024-02-09   31  2129240               23   

   LoanAmount  LoanTermMonths    Purpose  LoanToIncome  MissedPaymentsCount  \
0      499177              48   Personal        0.2264                    1   
1      101295              24       Home        0.0343                    1   
2      950519              24  Education        0.5970                    1   
3     1381666              48       Home        0.5464                    1   
4      657564              36  Education        0.3088                    1   

  Last

In [43]:
# Step 2: Basic cleaning & target
# -------------------------
# Working target: loan is 'delinquent' if MissedPaymentsCount >= 1
# This is a business proxy — adjust as needed.
df['target_default'] = (df['MissedPaymentsCount'].fillna(0) >= 1).astype(int)

In [45]:
# Feature ideas: Age, Income, EmploymentYears, LoanAmount, LoanTermMonths, LoanToIncome, MissedPaymentsCount, DaysSinceLastPayment
# Create DaysSinceLastPayment (large number if NULL)
today = pd.to_datetime(datetime.utcnow().date())
df['LastPaymentDate'] = pd.to_datetime(df['LastPaymentDate'])
df['DaysSinceLastPayment'] = (today - df['LastPaymentDate']).dt.days.fillna(9999).astype(int)

C:\Users\RAM\AppData\Local\Temp\ipykernel_21084\234430527.py:3: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  today = pd.to_datetime(datetime.utcnow().date())


In [47]:
# Keep a handful of features (you can expand)
features = [
    'Age',
    'Income',
    'EmploymentYears',
    'LoanAmount',
    'LoanTermMonths',
    'LoanToIncome',
    'MissedPaymentsCount',   # useful but correlated with target in our proxy
    'DaysSinceLastPayment',
    'Purpose'                # categorical
]

In [49]:
# Select usable rows (drop weird nulls)
df_model = df[features + ['ApplicationID', 'CustomerID', 'ApplicationDate', 'target_default']].copy()
df_model = df_model.dropna(subset=['ApplicationID', 'CustomerID', 'ApplicationDate'])
print("Model dataset rows:", len(df_model))

Model dataset rows: 1000


In [51]:
# Step 3: Train/test split
# -------------------------
X = df_model[features]
y = df_model['target_default']

In [57]:
# Simple train-test split (stratify to keep class ratio)
X_train, X_test, y_train, y_test, meta_train, meta_test = train_test_split(
    X, y, df_model[['ApplicationID', 'CustomerID', 'ApplicationDate']],
    test_size=0.2, random_state=42, stratify=y if y.nunique() > 1 else None
)
print("Train rows:", len(X_train), "Test rows:", len(X_test))

Train rows: 800 Test rows: 200


In [61]:
# Step 4: Preprocessing pipeline (robust to sklearn versions)
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import inspect

# features
num_features = ['Age','Income','EmploymentYears','LoanAmount','LoanTermMonths','LoanToIncome','MissedPaymentsCount','DaysSinceLastPayment']
cat_features = ['Purpose']

# numeric transformer
num_transformer = make_pipeline(
    SimpleImputer(strategy='median')
)

# detect which kwarg OneHotEncoder accepts ('sparse' or 'sparse_output')
ohe_kwargs = {}
sig = inspect.signature(OneHotEncoder)
if 'sparse_output' in sig.parameters:
    ohe_kwargs['sparse_output'] = False
else:
    # older sklearn
    ohe_kwargs['sparse'] = False

# categorical transformer
cat_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='Missing'),
    OneHotEncoder(handle_unknown='ignore', **ohe_kwargs)
)

# ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ],
    remainder='drop'
)

# Quick smoke-check (optional): fit_transform on a small slice to ensure shape is OK
# Only run this if X_train exists; otherwise skip
try:
    sample = X_train.head(5)
    ft = preprocessor.fit_transform(sample)
    print("Preprocessor output shape (sample):", ft.shape)
except NameError:
    print("X_train not found — re-run the data-prep cells before running this check.")


Preprocessor output shape (sample): (5, 12)


In [67]:
# Step 5: Train RandomForest
# -------------------------
clf = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)

pipe = make_pipeline(preprocessor, clf)

print("Training RandomForest pipeline ...")
pipe.fit(X_train, y_train)
print("Training complete.")


Training RandomForest pipeline ...
Training complete.


In [69]:
# Save model artifact (self-contained)
import os
import joblib
from datetime import datetime

# model pipeline 'pipe' must already exist in this kernel (trained)
# If pipe is not defined, re-run the training cell first.

# Create a timestamped model version and safe file path
MODEL_VERSION = f"rf_baseline_v1_{datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}"
MODELS_DIR = "models"
os.makedirs(MODELS_DIR, exist_ok=True)
MODEL_FILE = os.path.join(MODELS_DIR, f"loan_model_{MODEL_VERSION}.joblib")

# Save
joblib.dump(pipe, MODEL_FILE)
print(f"Saved model to: {MODEL_FILE}")
print(f"MODEL_VERSION = '{MODEL_VERSION}'")


Saved model to: models\loan_model_rf_baseline_v1_20251011T083235Z.joblib
MODEL_VERSION = 'rf_baseline_v1_20251011T083235Z'


C:\Users\RAM\AppData\Local\Temp\ipykernel_21084\473293729.py:10: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  MODEL_VERSION = f"rf_baseline_v1_{datetime.utcnow().strftime('%Y%m%dT%H%M%SZ')}"


In [71]:
# Step 6: Evaluate on test set
# -------------------------
y_pred = pipe.predict(X_test)
y_proba = pipe.predict_proba(X_test)[:,1] if hasattr(pipe, "predict_proba") else pipe.decision_function(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred, zero_division=0)
rec = recall_score(y_test, y_pred, zero_division=0)
f1 = f1_score(y_test, y_pred, zero_division=0)
auc = roc_auc_score(y_test, y_proba) if len(np.unique(y_test))>1 else np.nan

print("Evaluation (test):")
print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {rec:.4f}, F1: {f1:.4f}, AUC: {auc:.4f}")
print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

Evaluation (test):
Accuracy: 1.0000, Precision: 1.0000, Recall: 1.0000, F1: 1.0000, AUC: 1.0000
Confusion matrix:
[[102   0]
 [  0  98]]


In [73]:
# Step 7: Prepare predictions for all data (or a selection)
# -------------------------
# Predict probabilities for all rows in df_model (you could limit to recent loans)
X_all = df_model[features]
probs_all = pipe.predict_proba(X_all)[:,1] if hasattr(pipe, "predict_proba") else pipe.decision_function(X_all)
labels_all = (probs_all >= 0.5).astype(int)  # threshold 0.5 — tune as needed

preds = df_model[['ApplicationID', 'CustomerID', 'ApplicationDate']].copy()
preds['Pred_Prob'] = probs_all
preds['Pred_Label'] = labels_all
preds['ModelVersion'] = MODEL_VERSION
preds['ModelRunDate'] = datetime.utcnow()

print("Prepared predictions for", len(preds), "rows")

Prepared predictions for 1000 rows


C:\Users\RAM\AppData\Local\Temp\ipykernel_21084\1965800682.py:12: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  preds['ModelRunDate'] = datetime.utcnow()


In [81]:
# Minimal: define variable first (recommended)
STAGING_TABLE = 'dbo.LoanPredictions_Staging'   # <- define this before using it

create_staging_sql = f"""
IF OBJECT_ID('{STAGING_TABLE}') IS NULL
BEGIN
    CREATE TABLE {STAGING_TABLE} (
        ApplicationID INT,
        CustomerID INT,
        ApplicationDate DATE,
        Pred_Prob FLOAT,
        Pred_Label BIT,
        ModelVersion VARCHAR(100),
        ModelRunDate DATETIME2
    );
END;
"""
with engine.begin() as conn:
    conn.execute(text(create_staging_sql))
    # Clear staging
    conn.execute(text(f"DELETE FROM {STAGING_TABLE};"))

# Pandas to_sql: note use fast_executemany via the engine created earlier
preds.to_sql(name=STAGING_TABLE.split('.')[-1], schema=STAGING_TABLE.split('.')[0],
             con=engine, if_exists='append', index=False, method=None)
print("Wrote predictions to staging:", STAGING_TABLE)


Wrote predictions to staging: dbo.LoanPredictions_Staging


In [85]:
# Step 9: MERGE into target LoanPredictions table
# -------------------------
TARGET_TABLE = globals().get('TARGET_TABLE', 'dbo.LoanPredictions')
merge_sql = f"""
MERGE INTO {TARGET_TABLE} AS Target
USING {STAGING_TABLE} AS Src
  ON Target.ApplicationID = Src.ApplicationID AND Target.ModelVersion = Src.ModelVersion
WHEN MATCHED THEN
  UPDATE SET
    Target.Pred_Prob = Src.Pred_Prob,
    Target.Pred_Label = Src.Pred_Label,
    Target.ModelRunDate = Src.ModelRunDate
WHEN NOT MATCHED BY TARGET THEN
  INSERT (ApplicationID, CustomerID, ApplicationDate, Pred_Prob, Pred_Label, ModelVersion, ModelRunDate)
  VALUES (Src.ApplicationID, Src.CustomerID, Src.ApplicationDate, Src.Pred_Prob, Src.Pred_Label, Src.ModelVersion, Src.ModelRunDate);
"""
with engine.begin() as conn:
    conn.execute(text(merge_sql))
print("MERGE completed into", TARGET_TABLE)



MERGE completed into dbo.LoanPredictions


In [87]:
# Step 10: Quick verification from DB
# -------------------------
with engine.connect() as conn:
    res = pd.read_sql("SELECT TOP (10) * FROM dbo.LoanPredictions ORDER BY ModelRunDate DESC;", conn)
    print("Top LoanPredictions rows:")
    print(res.head(10))

print("Pipeline finished. Model version:", MODEL_VERSION)

Top LoanPredictions rows:
   PredictionID  ApplicationID  CustomerID ApplicationDate  Pred_Prob  \
0            11           2011        1118      2024-06-13   1.000000   
1            10           2010        1135      2022-04-21   0.020000   
2             9           2009        1289      2023-10-02   0.993333   
3             8           2008        1319      2023-01-21   0.980000   
4             7           2007        1257      2022-04-29   0.026667   
5             6           2006        1458      2023-09-10   0.993333   
6             5           2005        1406      2024-02-09   1.000000   
7             4           2004        1380      2024-10-05   1.000000   
8             3           2003        1333      2023-05-19   0.993333   
9             2           2002        1124      2024-09-18   0.993333   

   Pred_Label                     ModelVersion               ModelRunDate  
0        True  rf_baseline_v1_20251011T083235Z 2025-10-11 08:34:31.550733  
1       False  rf_